### Actions
N(0), E(1), S(2), W(3)

### States
N(0), NE(1), E(2), SE(3), S(4), SW(5), W(6), NW(7)


### Rewards
1. +10 Eats Food
2. -0.1 Moves

In [1]:
from matplotlib import pyplot as plt
import matplotlib.animation
import random
import numpy as np
import json

In [2]:
WIDTH = 4
HEIGHT = 4

In [11]:
def get_possible_actions(current_pos):    
    return [0,1,2,3]

def get_next_pos(current_pos, action):
    if(action == 0):
        return [current_pos[0] - 1,current_pos[1]] 
    elif (action == 2):
        return [current_pos[0] + 1,current_pos[1]] 
    elif (action == 1):
        return [current_pos[0],current_pos[1]+1]
    else:
        return [current_pos[0],current_pos[1]-1]


def get_state(current_pos, food_pos):
# NW(0), NW&EW(1), EW(2), EW&SW(3) SW(4),SW&WW(5) WW(6) WW&NW(7), NODANGER(8)
# N(0), NE(1), E(2), SE(3), S(4), SW(5), W(6), NW(7), FOODATCURRENTPOS(8)
    state = [8,8]

    
    if current_pos[0]-1 == -1 and current_pos[1]+1 != WIDTH and current_pos[1]-1 != -1:
        state[0] = 0
    elif current_pos[0]-1 == -1 and current_pos[1]+1 == WIDTH:
        state[0] = 1
    elif current_pos[1]+1 == WIDTH and current_pos[0]+1 != HEIGHT and current_pos[0]-1 != -1: 
        state[0] = 2
    elif current_pos[1]+1 == WIDTH and current_pos[0]+1 == HEIGHT: 
        state[0] = 3 
    elif current_pos[0]+1 == HEIGHT and current_pos[1]+1 != WIDTH and current_pos[1]-1 != -1: 
        state[0] = 4
    elif current_pos[0]+1 == HEIGHT and current_pos[1]-1 == -1: 
        state[0] = 5
    elif current_pos[1]-1 == -1 and current_pos[0]+1 != HEIGHT and current_pos[0]-1 != -1: 
        state[0] = 6
    elif current_pos[0]-1 == -1 and current_pos[0]-1 == -1:
        state[0] = 7
         

    if current_pos[0] > food_pos[0] and current_pos[1] == food_pos[1]:
        state[1]=0
    elif current_pos[0] > food_pos[0] and current_pos[1] < food_pos[1]:
        state[1]=1
    elif current_pos[0] == food_pos[0] and current_pos[1] < food_pos[1]:
        state[1]=2
    elif current_pos[0] < food_pos[0] and current_pos[1] < food_pos[1]:
        state[1]=3
    elif current_pos[0] < food_pos[0] and current_pos[1] == food_pos[1]:
        state[1]=4
    elif current_pos[0] < food_pos[0] and current_pos[1] > food_pos[1]:
        state[1]=5
    elif current_pos[0] == food_pos[0] and current_pos[1] > food_pos[1]:
        state[1]=6
    elif current_pos[0] > food_pos[0] and current_pos[1] > food_pos[1]:
        state[1]=7
    return state 

def game_complete(current_pos):
    return current_pos[0] < 0 or current_pos[0] >= HEIGHT or current_pos[1] < 0 or current_pos[0] >= WIDTH 

def set_food_pos(current_pos):
    food_pos = [random.randint(0, HEIGHT-1), random.randint(0, WIDTH-1)]
    while food_pos == current_pos:
        food_pos = [random.randint(0, HEIGHT-1), random.randint(0, WIDTH-1)]
    return food_pos

def evaluate_move(current_pos, food_pos):
    if current_pos == food_pos:
        return 10000
    elif game_complete(current_pos):
        return -100
    else: 
        return -1


In [26]:
get_state([0,0],[3,3])

[7, 3]

In [4]:
def generate_env_q_matrix(height, width):
    env = [[0 for x in range(width)] for y in range(height)]
    
    # NW(0), EW(1), SW(2), WW(3)
    # N(0), NE(1),E(2), SE(3), S(4), SW(5), W(6), NW(7)
    q_matrix = [[[0,0,0,0] for x in range(9)] for y in range(9)]
    
    
    return env, q_matrix

In [22]:
discount = 0.9
learning_rate = 0.1
env, q_matrix = generate_env_q_matrix(HEIGHT, WIDTH)
_, count = generate_env_q_matrix(HEIGHT, WIDTH)
scores = [0]


for _ in range(1000000):
    current_pos = [random.randint(0, HEIGHT-1), random.randint(0, WIDTH-1)]
    food_pos = set_food_pos(current_pos)
    points = 0
 

    while(not game_complete(current_pos)):
     
        

        state = get_state(current_pos, food_pos)
       
        possible_actions = get_possible_actions(current_pos)
        action = random.choice(possible_actions)
        next_pos = get_next_pos(current_pos, action)
        reward = evaluate_move(next_pos, food_pos)
        if reward == 10000:
            points+=1
       
      

        
        q_matrix[state[0]][state[1]][action] += learning_rate * (reward + (discount * max(q_matrix[state[0]][state[1]]))) - q_matrix[state[0]][state[1]][action]
        
        count[state[0]][state[1]][action] += 1
        
        if state[1] == 8:
            food_pos = set_food_pos(current_pos)

        current_pos = next_pos

        discount -= discount/10000000
        learning_rate +=  learning_rate/10000000
    
    if points>scores[-1]:
        scores.append(points)
      
    






In [23]:
values = []
directions = []
for row in q_matrix:
    for column in row:
        values.append(column[np.argmax(column)])
        directions.append(np.argmax(column))
        
values = np.reshape(values,(9,9))
directions = np.reshape(directions,(9,9))

print(directions)
print(discount, learning_rate)
print(scores)

[[0 0 1 3 2 2 3 0 3]
 [0 0 0 0 3 2 3 0 1]
 [0 0 0 0 0 3 3 2 0]
 [0 0 0 0 0 0 0 3 3]
 [1 3 1 0 0 0 3 1 3]
 [0 0 1 0 0 0 0 0 1]
 [0 2 1 1 2 0 0 0 0]
 [0 0 2 1 2 0 0 0 1]
 [0 3 1 3 2 2 1 3 1]]
0.4341516195422779 0.20730083544063252
[0, 2, 3, 4, 5, 6, 7]


In [24]:
env, _ = generate_env_q_matrix (HEIGHT ,WIDTH)

current_pos = [random.randint(0, HEIGHT-1), random.randint(0, WIDTH-1)]
env[current_pos[0]][current_pos[1]] = 50

food_pos = [3,3]
env[food_pos[0]][food_pos[1]] = 100


fig, ax = plt.subplots()
l = ax.imshow(env)

def move():
 
    global current_pos
    global food_pos

    state = get_state(current_pos, food_pos)

    action = directions[state[0]][state[1]]

    next_pos =  get_next_pos(current_pos, action)
    
    env[current_pos[0]][current_pos[1]] = 0

    current_pos = next_pos

    env[current_pos[0]][current_pos[1]] = 50


    if state[1] == 8:
        env[food_pos[0]][food_pos[1]] = 0
        food_pos = set_food_pos(current_pos)
        env[food_pos[0]][food_pos[1]] = 100
    
def anim(i):
    l.set_data(env)
    move()


ani = matplotlib.animation.FuncAnimation(fig, anim)


plt.close()
from IPython.display import HTML
HTML(ani.to_jshtml())